In [6]:
# Import necessary libraries
import visualkeras
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
# Define constants
NUM_CLASSES = 10
IMG_SIZE = 64

In [3]:
# 1. 1D CNN model
def build_1d_cnn(input_shape=(300,1), num_classes=NUM_CLASSES):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv1D(16, 9, activation='relu', padding='same')(inp)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(32, 7, activation='relu', padding='same')(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(64, 5, activation='relu', padding='same')(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inp, out)

# 2.) 2D CNN Model
def build_2d_cnn(input_shape=(IMG_SIZE, IMG_SIZE, 1), num_classes=NUM_CLASSES):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inp)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inp, out)

# 3.) Hybrid Model
def build_1d_branch(input_shape=(300,)):
    inp = layers.Input(shape=input_shape, name="ecg_signal")
    x = layers.Reshape((input_shape[0], 1))(inp)

    for filters, k, pool in [(32, 7, 2), (64, 5, 2), (128, 5, None), (256, 3, None)]:
        x = layers.Conv1D(filters, kernel_size=k, padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        if pool:
            x = layers.MaxPool1D(pool_size=pool)(x)
        x = layers.Dropout(0.2)(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    model = models.Model(inputs=inp, outputs=x, name='1d_branch')
    return model

def conv2d_block(x, filters, kernel_size=3, pool=True):
    x = layers.Conv2D(filters, kernel_size, padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, kernel_size, padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    if pool:
        x = layers.MaxPool2D(pool_size=2)(x)
    x = layers.Dropout(0.25)(x)
    return x

def build_2d_branch(input_shape=(IMG_SIZE, IMG_SIZE, 1)):
    inp = layers.Input(shape=input_shape, name='ecg_image')
    x = inp
    for f in [32, 64, 128]:
        x = conv2d_block(x, f, kernel_size=3, pool=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    model = models.Model(inputs=inp, outputs=x, name='2d_branch')
    return model

def build_hybrid_model(signal_shape=(300,), image_shape=(IMG_SIZE, IMG_SIZE, 1), num_classes=NUM_CLASSES):
    branch1 = build_1d_branch(signal_shape)
    branch2 = build_2d_branch(image_shape)

    combined = layers.concatenate([branch1.output, branch2.output])
    x = layers.Dense(256, activation='relu')(combined)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)

    out = layers.Dense(num_classes, activation='softmax', name='output')(x)
    model = models.Model(inputs=[branch1.input, branch2.input], outputs=out, name='ecg_hybrid')
    return model

In [8]:
# Create and visualize the models
print("Generating visualizations...")

# 1D CNN
model_1d = build_1d_cnn()
visualkeras.layered_view(model_1d, to_file='1d_cnn_visual.png', legend=True, draw_funnel=True, spacing=10)
print("1D CNN visualization saved to 1d_cnn_visual.png")

# 2D CNN
model_2d = build_2d_cnn()
visualkeras.layered_view(model_2d, to_file='2d_cnn_visual.png', legend=True, draw_funnel=True, spacing=10)
print("2D CNN visualization saved to 2d_cnn_visual.png")

# Hybrid CNN
model_hybrid = build_hybrid_model()
visualkeras.layered_view(model_hybrid, to_file='hybrid_cnn_visual.png', legend=True, draw_funnel=True, spacing=10)
print("Hybrid CNN visualization saved to hybrid_cnn_visual.png")

Generating visualizations...
1D CNN visualization saved to 1d_cnn_visual.png
2D CNN visualization saved to 2d_cnn_visual.png


c:\Users\LENOVO\miniconda3\envs\ecg_cycu_project\Lib\site-packages\visualkeras\layered.py:86: UserWarning: The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.
  warnings.warn("The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.")


Hybrid CNN visualization saved to hybrid_cnn_visual.png
